In [29]:
import numpy as np
import sys 
import os
import matplotlib.pyplot as plt
import copy 
import tqdm
import cv2
import pandas as pd

In [2]:
distance = '200Mpc/'
box_list = ['box01/','box02/']
subbox_list = ['1','2','3','4','5','6','7','8']
ref_path = '/storage/filament/data/' + distance


volume_const  = 0.01*( (200/2048)*(3.086/0.7))**3

for box_num in box_list:
    box_path = '/storage/filament/data/' + distance + box_num
    os.chdir(box_path)
    for subbox_num in subbox_list:
        print("box ",box_num, "subbox num",subbox_num)
        xray = np.fromfile(box_path + 'sub0' + subbox_num + 'X1024' ,dtype='float32')
        xray = xray.reshape([1024,1024,1024])
        xray =  xray*volume_const

        temp = np.fromfile(box_path + 'sub0' + subbox_num + 'T1024' ,dtype='float32')
        temp = temp.reshape([1024,1024,1024])

        dens = np.fromfile(box_path + 'sub0' + subbox_num + 'd1024' ,dtype='float32')
        dens = dens.reshape([1024,1024,1024])


        candidate_coordx = []
        candidate_coordy = []
        candidate_coordz = []
        nx = 1024
        for iz in range(30,nx-30):
            for iy in range(30,nx-30):
                for ix in range(30,nx-30):
                    if xray[ix,iy,iz] >= 10**(-5.5):
                        virial_radius = 0
                        # virial raius 계산
                        for vr in range(30):
                            critical_density = np.mean(dens[ix-vr:ix+vr , iy-vr:iy+vr , iz-vr:iz+vr])
                            #print(critical_density)
                            if (critical_density < 200):
                                virial_radius = vr
                                break
                            else:
                                pass

                        # 주변 local maximun tag
                        if not virial_radius == 0:
                            ref_value = xray[ix,iy,iz]
                            tmp_x = ix
                            tmp_y = iy
                            tmp_z = iz
                            max_value = ref_value
                            for rx in range(-virial_radius,virial_radius+1):
                                for ry in range(-virial_radius,virial_radius+1):
                                    for rz in range(-virial_radius,virial_radius+1):
                                        if ref_value < xray[ix+rx, iy+ry, iz+rz]:
                                            tmp_x = 0
                                            tmp_y = 0
                                            tmp_z = 0


                                        else:
                                            pass


                        # xray weighted temperature
                        xray_sum = 0
                        xray_temp_sum = 0
                        if  tmp_x !=0 and tmp_y != 0 and tmp_z !=0:
                            for iix in range(-virial_radius, virial_radius+1):
                                for iiy in range(-virial_radius, virial_radius+1):
                                    for iiz in range(-virial_radius, virial_radius+1):
                                        xray_sum = xray_sum + xray[tmp_x+iix,tmp_y+iiy,tmp_z+iiz]
                                        xray_temp_sum = xray_temp_sum + xray[tmp_x+iix,tmp_y+iiy,tmp_z+iiz]*temp[tmp_x+iix,tmp_y+iiy,tmp_z+iiz]

                            xray_temp = xray_temp_sum/xray_sum
                            
                            if xray_temp >= 2*1.16*10**7:
                                print("confirmed",ix,iy,iz)
                                candidate_coordx.append(tmp_x)
                                candidate_coordy.append(tmp_y)
                                candidate_coordz.append(tmp_z)
                            else:
                                continue

        df = pd.DataFrame({'x':candidate_coordx,'y':candidate_coordy,'z':candidate_coordz})
        coords_path = '/storage/filament/data/200Mpc/' + box_num
        df.to_excel(coords_path + subbox_num + '.xlsx')

box  box01/ subbox num 1


/home/ha/anaconda3/lib/python3.7/site-packages/numpy/core/fromnumeric.py:2920: RuntimeWarning: Mean of empty slice.
  out=out, **kwargs)
/home/ha/anaconda3/lib/python3.7/site-packages/numpy/core/_methods.py:85: RuntimeWarning: invalid value encountered in true_divide
  ret = ret.dtype.type(ret / rcount)


confirmed 123 438 153
confirmed 292 607 340
confirmed 504 653 395
confirmed 462 233 422
confirmed 594 989 424
confirmed 517 296 425
confirmed 436 500 452
confirmed 438 505 454
confirmed 451 503 457
confirmed 369 132 462
confirmed 433 497 465
confirmed 149 417 467
confirmed 359 136 469
confirmed 445 503 476
confirmed 380 454 479
confirmed 452 458 479
confirmed 381 366 550
confirmed 525 492 623
confirmed 263 99 897
box  box01/ subbox num 2
confirmed 451 552 109
confirmed 189 814 294
confirmed 252 768 391
confirmed 251 485 422
confirmed 467 79 719
confirmed 693 585 752
confirmed 125 96 854
confirmed 320 198 903
confirmed 177 143 906
confirmed 185 142 920
confirmed 174 128 921
confirmed 199 147 921
confirmed 189 129 925
confirmed 198 149 926
confirmed 195 153 926
box  box01/ subbox num 3
confirmed 689 555 205
confirmed 588 679 490
confirmed 585 676 491
confirmed 594 672 498
confirmed 592 687 499
confirmed 590 668 515
confirmed 601 674 515
confirmed 588 680 515
confirmed 228 590 553
confirm

In [37]:
sub01 = [0,0,0]
sub02 = [1024,1024,1024] 
sub03 = [0,1024,1024]
sub04 = [0,0,1024]
sub05 = [0,1024,0]
sub06 = [1024,0,0]
sub07 = [1024,0,1024]
sub08 = [1024,1024,0]

In [2]:
subbox_num = '7'
distance= '200Mpc/'
ref_path = '/storage/filament/data/' + distance
box_path = ref_path + 'box02/'

#coords = np.array(pd.read_csv(box_path + subbox_num + '.csv'))
dens = np.fromfile(box_path + 'sub0' + subbox_num + 'd1024' ,dtype='float32')
dens = dens.reshape([1024,1024,1024])

In [31]:
tmp_x = coords[0][1]
tmp_y = coords[0][2]
tmp_z = coords[0][3]

In [39]:
ref_x = tmp_x + sub08[0]
ref_y = tmp_y + sub08[1]
ref_z = tmp_z + sub08[2]

In [5]:
i = 261
j = 448
k = 775

In [6]:
dens_test = np.zeros([401,401,401])

In [ ]:
dens_test = dens[i-200:i+200+1,j-200:j+200+1,k-200:k+200+1]

In [8]:
dens_test[200,200,200]

10536.97

In [20]:
dens_mat_test = []
for i in range(0,401,4):
    for j in range(0,401,4):
        for k in range(0,401,4):
                dens_mat_test.append(dens_test[i,j,k])

In [22]:
dens_mat_test_re = np.array(dens_mat_test).reshape([101,101,101])

In [ ]:
os.chdir('/storage/filament/data/200Mpc/box02/pic_test/')
for nn in range(101):
    plt.figure(figsize=[30,30])
    plt.imsave(str(nn) +'.png',np.log10(dens_mat_test_re[:,:,nn]),vmin=0,vmax=4,cmap='gray',dpi=300,format='png')

In [28]:
os.chdir('/storage/filament/data/200Mpc/box02/pic_test/full/')
for nn in range(401):
    plt.figure(figsize=[30,30])
    plt.imsave(str(nn) +'.png',np.log10(dens_test[:,:,nn]),vmin=0,vmax=4,cmap='gray',dpi=300,format='png')

/home/ha/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:3: RuntimeWarning: More than 20 figures have been opened. Figures created through the pyplot interface (`matplotlib.pyplot.figure`) are retained until explicitly closed and may consume too much memory. (To control this warning, see the rcParam `figure.max_open_warning`).
  This is separate from the ipykernel package so we can avoid doing imports until


<Figure size 2160x2160 with 0 Axes>

<Figure size 2160x2160 with 0 Axes>

<Figure size 2160x2160 with 0 Axes>

<Figure size 2160x2160 with 0 Axes>

...

<Figure size 2160x2160 with 0 Axes>

<Figure size 2160x2160 with 0 Axes>

<Figure size 2160x2160 with 0 Axes>

<Figure size 2160x2160 with 0 Axes>

<Figure size 2160x2160 with 0 Axes>

In [2]:
dens = np.fromfile(box_path + 'sub0' + subbox_num + 'd1024' ,dtype='float32')
dens = dens.reshape([1024,1024,1024])

In [ ]:
distance = '200Mpc/'
box_list = ['box01/','box02/']
subbox_list = ['1','2','3','4','5','6','7','8']
ref_path = '/storage/filament/data/' + distance

dens = np.fromfile(box_path + 'sub0' + subbox_num + 'd1024' ,dtype='float32')
dens = dens.reshape([1024,1024,1024])

In [11]:
dens[974,831,122]

i=766
j=918
k=458

In [12]:
dens_test = dens[i-200:i+200,j-200:j+200,k-200:k+200]

In [13]:
dens_test.shape

(400, 306, 400)

In [ ]:
distance == '200Mpc/'
box_list = ['box01/','box02/']
subbox_list = ['1','2','3','4','5','6','7','8']
ref_path = '/storage/filament/data/' + distance

for box_num in box_list:
    box_path = ref_path + box_num
    os.chdir(box_path)
    for subbox_num in subbox_list:
        dens = np.fromfile(box_path + 'sub0' + subbox_num + 'd1024' ,dtype='float32')
        dens = dens.reshape([1024,1024,1024])

In [ ]:
box_num = '02'
#subbox_num = ''

subbox_list = ['1','2','3','4','5','6','7','8']

distance = '40Mpc'

if distance == '30Mpc':
    box_size = 37
if distance == '40Mpc':
    box_size = 50

for subbox_num in subbox_list:
    data_path = '/storage/filament/data/box'+box_num +'/'
    dens_result_path = '/storage/filament/result/cluster_3d/' + distance + '/density_log_plot/box' + box_num + '/subbox0' +subbox_num + '/'
    dens_smooth_path_5 =  '/storage/filament/result/cluster_3d/' + distance + '/density_smoothing_plot_5_v2/box' + box_num + '/subbox0' +subbox_num + '/'
    data_type = ['a','f','g']
    # a = density
    # f = temperature
    # g = xray   
    
    dens = np.zeros(256**3,dtype='float64')
    dens = np.genfromtxt(data_path + '256den18' + data_type[0] + '+1024+' + subbox_num,dtype='float64')
    dens_mat = dens.reshape([256,256,256])

    temp = np.zeros(256**3,dtype='float64')
    temp = np.genfromtxt(data_path + '256den18' + data_type[1] + '+1024+' + subbox_num,dtype='float64')
    temp_mat = temp.reshape([256,256,256])

    index_path = '/storage/filament/data/box'+box_num +'+Cinfo/'
    index = np.genfromtxt(index_path + 'box' + box_num + '+xlum+' + subbox_num+'.dat',dtype='uint8')

    x_index = index[:,3]-1
    y_index = index[:,2]-1
    z_index = index[:,1]-1

    coords = np.array(list(zip(x_index,y_index,z_index)))
    coords = np.array(sorted(coords, key=lambda x:x[2] ))

    box_list =[]
    temp_point = np.zeros([box_size*2,box_size*2,box_size*2],dtype='float64')
    dens_3d = np.zeros([box_size*2,box_size*2,box_size*2],dtype='float64')
    smooth_dens = np.zeros([box_size*2,box_size*2,box_size*2],dtype='float64')
    
    cluster_peak_thres = 2*1.16*10**7
               
    min_thres = 0.5
    max_thres = 0.7
    min_thres_2 = 1.0
    max_thres_2 = 1.5
    
    
    num=0

    print('box_num ', box_num, 'subbox_num ',subbox_num, 'density')

    for n,(ix,iy,iz) in enumerate(tqdm.tqdm(coords)):
        if (ix < box_size or ix>256-box_size or iy < box_size or iy > 256-box_size or iz <box_size or iz >256-box_size):
            continue
        else:
            if temp_mat[ix,iy,iz] >= cluster_peak_thres:
                #print('box_num ', box_num, 'subbox_num ',subbox_num, 'density',ix,iy,iz)
                dens_3d[:,:,:] = dens_mat[-box_size+ix:box_size+ix,-box_size+iy:box_size+iy,-box_size+iz:box_size+iz]

               
                smooth_dens = smooting_3d(dens_3d,4)
                smooth_dens = smooth_dens[2:100-2,2:100-2,2:100-2]
                
                if not os.path.isdir(dens_smooth_path_5 + str(num) + '/'):
                     os.makedirs(dens_smooth_path_5 + str(num) + '/')
                os.chdir(dens_smooth_path_5 + str(num) + '/')            

                for nn in range(2*box_size-4):
                    plt.figure(figsize=[30,30])
                    plt.imsave(str(nn) +'.png',np.log10(smooth_dens[:,:,nn]),vmin=-1,vmax=1,cmap='gray',dpi=300,format='png')

                    
                
                num = num+1
               
     

In [ ]:
def smooting_3d(array,smooth_size):
    array_len = array.shape[0]
    half_smooth_size = int(smooth_size/2)
    #smooth_array = np.zeros([array_len-smooth_size,array_len-smooth_size,array_len-smooth_size])
    smooth_array = np.zeros([array_len,array_len,array_len])
    for i in range(half_smooth_size,array_len-half_smooth_size):
        for j in range(half_smooth_size,array_len-half_smooth_size):
            for k in range(half_smooth_size,array_len-half_smooth_size):
                

                #smooth_array[i,j,k] = np.mean(array[i-half_smooth_size:i+half_smooth_size+1,j-half_smooth_size:j+half_smooth_size+1,k-half_smooth_size:k+half_smooth_size+1])
                smooth_array[i,j,k] = (1/(smooth_size+1)**3)*np.sum(array[i-half_smooth_size:i+half_smooth_size+1,j-half_smooth_size:j+half_smooth_size+1,k-half_smooth_size:k+half_smooth_size+1])    
    
    #print("b",smooth_array.shape)
    return smooth_array

### add_box에 있는것들!!

In [ ]:
box_size=50
box_name = 'box02_add'
data_path = '/storage/filament/data/'
data_path = data_path+box_name +'/box02_norm/'

file_list = np.sort(os.listdir(data_path))

dens_smooth_path_5 = '/storage/filament/result/cluster_3d/40Mpc/density_smooting_plot_5_v2/'
dens_smooth_path_5 = dens_smooth_path_5 + box_name +'/'

#for nn,data_name in enumerate(file_list:

#data_name = 'box01sub01C01.dat'
for data_name in file_list:
    dens = np.zeros(101**3,dtype='float64')
    dens = np.genfromtxt(data_path + data_name ,dtype='float64')

    dens_3d = dens.reshape([101,101,101])
    dens_3d = dens_3d[0:100 ,0:100, 0:100]


    smooth_dens = smooting_3d(dens_3d,4)
    smooth_dens = smooth_dens[2:100-2,2:100-2,2:100-2]

    if not os.path.isdir(dens_smooth_path_5 + str(data_name[:-4]) + '/'):
         os.makedirs(dens_smooth_path_5 + str(data_name[:-4]) + '/')
    os.chdir(dens_smooth_path_5 + str(data_name[:-4]) + '/')            

    for nn in range(2*box_size-4):
        plt.figure(figsize=[30,30])
        plt.imsave(str(nn) +'.png',np.log10(smooth_dens[:,:,nn]),vmin=-1,vmax=1,cmap='gray',dpi=300,format='png')

In [ ]:
box_num = '01'
#subbox_num = ''

subbox_list = ['8']

distance = '40Mpc'

if distance == '30Mpc':
    box_size = 37
if distance == '40Mpc':
    box_size = 50

for subbox_num in subbox_list:
    data_path = '/storage/filament/data/box'+box_num +'/'
    dens_result_path = '/storage/filament/result/cluster_3d/' + distance + '/density_log_plot/box' + box_num + '/subbox0' +subbox_num + '/'
    dens_smooth_path_5 =  '/storage/filament/result/cluster_3d/' + distance + '/density_smooting_plot_5_v2/box' + box_num + '/subbox0' +subbox_num + '/'
    data_type = ['a','f','g']
    # a = density
    # f = temperature
    # g = xray   
    
    dens = np.zeros(256**3,dtype='float64')
    dens = np.genfromtxt(data_path + '256den18' + data_type[0] + '+1024+' + subbox_num,dtype='float64')
    dens_mat = dens.reshape([256,256,256])

    temp = np.zeros(256**3,dtype='float64')
    temp = np.genfromtxt(data_path + '256den18' + data_type[1] + '+1024+' + subbox_num,dtype='float64')
    temp_mat = temp.reshape([256,256,256])

    index_path = '/storage/filament/data/box'+box_num +'+Cinfo/'
    index = np.genfromtxt(index_path + 'box' + box_num + '+xlum+' + subbox_num+'.dat',dtype='uint8')

    x_index = index[:,3]-1
    y_index = index[:,2]-1
    z_index = index[:,1]-1

    coords = np.array(list(zip(x_index,y_index,z_index)))
    coords = np.array(sorted(coords, key=lambda x:x[2] ))

    box_list =[]
    temp_point = np.zeros([box_size*2,box_size*2,box_size*2],dtype='float64')
    dens_3d = np.zeros([box_size*2,box_size*2,box_size*2],dtype='float64')
    smooth_dens = np.zeros([box_size*2,box_size*2,box_size*2],dtype='float64')
    
    cluster_peak_thres = 2*1.16*10**7
               
    min_thres = 0.5
    max_thres = 0.7
    min_thres_2 = 1.0
    max_thres_2 = 1.5
    
    
    num=0

    print('box_num ', box_num, 'subbox_num ',subbox_num, 'density')

    for n,(ix,iy,iz) in enumerate(tqdm.tqdm(coords)):
        if (ix < box_size or ix>256-box_size or iy < box_size or iy > 256-box_size or iz <box_size or iz >256-box_size):
            continue
        else:
            if temp_mat[ix,iy,iz] >= cluster_peak_thres:
                #print('box_num ', box_num, 'subbox_num ',subbox_num, 'density',ix,iy,iz)
                dens_3d[:,:,:] = dens_mat[-box_size+ix:box_size+ix,-box_size+iy:box_size+iy,-box_size+iz:box_size+iz]

               
                smooth_dens = smooting_3d(dens_3d,4)
                smooth_dens = smooth_dens[2:100-2,2:100-2,2:100-2]
                
                if not os.path.isdir(dens_smooth_path_5 + str(num) + '/'):
                     os.makedirs(dens_smooth_path_5 + str(num) + '/')
                os.chdir(dens_smooth_path_5 + str(num) + '/')            

                for nn in range(2*box_size-4):
                    plt.figure(figsize=[30,30])
                    plt.imsave(str(nn) +'.png',np.log10(smooth_dens[:,:,nn]),vmin=-1,vmax=1,cmap='gray',dpi=300,format='png')

                    
                
                num = num+1
               
     

In [ ]:
box_num = '01'
#subbox_num = ''

subbox_list = ['1','2','3','4','5','6','7','8']

distance = '40Mpc'

if distance == '30Mpc':
    box_size = 37
if distance == '40Mpc':
    box_size = 50

for subbox_num in subbox_list:
    data_path = '/storage/filament/data/200Mpc_v2/box'+box_num +'/'
    dens_result_path = '/storage/filament/result/cluster_3d/' + distance + '/density_log_plot/box' + box_num + '/subbox0' +subbox_num + '/'
    dens_smooth_path_5 =  '/storage/filament/result/cluster_3d/' + distance + '/density_log_plot/box' + box_num + '/subbox0' +subbox_num + '/'
    data_type = ['a','f','g']
    # a = density
    # f = temperature
    # g = xray   
    
    dens = np.zeros(256**3,dtype='float64')
    dens = np.genfromtxt(data_path + '256den18' + data_type[0] + '+1024+' + subbox_num,dtype='float64')
    dens_mat = dens.reshape([256,256,256])

    temp = np.zeros(256**3,dtype='float64')
    temp = np.genfromtxt(data_path + '256den18' + data_type[1] + '+1024+' + subbox_num,dtype='float64')
    temp_mat = temp.reshape([256,256,256])

    index_path = '/storage/filament/data/200Mpc_v2/box'+box_num +'+Cinfo/'
    index = np.genfromtxt(index_path + 'box' + box_num + '+xlum+' + subbox_num+'.dat',dtype='uint8')

    x_index = index[:,3]-1
    y_index = index[:,2]-1
    z_index = index[:,1]-1

    coords = np.array(list(zip(x_index,y_index,z_index)))
    coords = np.array(sorted(coords, key=lambda x:x[2] ))

    box_list =[]
    temp_point = np.zeros([box_size*2,box_size*2,box_size*2],dtype='float64')
    dens_3d = np.zeros([box_size*2,box_size*2,box_size*2],dtype='float64')
    smooth_dens = np.zeros([box_size*2,box_size*2,box_size*2],dtype='float64')
    
    cluster_peak_thres = 2*1.16*10**7
               
    min_thres = 0.5
    max_thres = 0.7
    min_thres_2 = 1.0
    max_thres_2 = 1.5
    
    
    num=0

    print('box_num ', box_num, 'subbox_num ',subbox_num, 'density')

    for n,(ix,iy,iz) in enumerate(tqdm.tqdm(coords)):
        if (ix < box_size or ix>256-box_size or iy < box_size or iy > 256-box_size or iz <box_size or iz >256-box_size):
            continue
        else:
            if temp_mat[ix,iy,iz] >= cluster_peak_thres:
                #print('box_num ', box_num, 'subbox_num ',subbox_num, 'density',ix,iy,iz)
                dens_3d[:,:,:] = dens_mat[-box_size+ix:box_size+ix,-box_size+iy:box_size+iy,-box_size+iz:box_size+iz]
                
                if not os.path.isdir(dens_smooth_path_5 + str(num) + '/'):
                     os.makedirs(dens_smooth_path_5 + str(num) + '/')
                os.chdir(dens_smooth_path_5 + str(num) + '/')            

                for nn in range(2*box_size-4):
                    plt.figure(figsize=[30,30])
                    plt.imsave(str(nn) +'.png',np.log10(dens_3d[:,:,nn]),vmin=0,vmax=4,cmap='gray',dpi=300,format='png')

                    
                
                num = num+1
               
     

  0%|          | 0/690 [00:00<?, ?it/s]

box_num  01 subbox_num  1 density


/home/ha/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:70: RuntimeWarning: More than 20 figures have been opened. Figures created through the pyplot interface (`matplotlib.pyplot.figure`) are retained until explicitly closed and may consume too much memory. (To control this warning, see the rcParam `figure.max_open_warning`).
  0%|          | 0/674 [00:00<?, ?it/s]

box_num  01 subbox_num  2 density


  0%|          | 0/674 [00:00<?, ?it/s]

...

  0%|          | 0/645 [00:00<?, ?it/s]

box_num  01 subbox_num  7 density


  0%|          | 0/672 [00:00<?, ?it/s]

box_num  01 subbox_num  8 density


100%|██████████| 672/672 [00:01<00:00, 594.67it/s]


In [ ]:
box_size=50
box_name = 'box02_add'
data_path = '/storage/filament/data/'
data_path = data_path+box_name +'/box02_norm/'

file_list = np.sort(os.listdir(data_path))

dens_smooth_path_5 = '/storage/filament/result/cluster_3d/40Mpc/density_log_plot/'
dens_smooth_path_5 = dens_smooth_path_5 + box_name +'/'

#for nn,data_name in enumerate(file_list:

#data_name = 'box01sub01C01.dat'
for data_name in file_list:
    dens = np.zeros(101**3,dtype='float64')
    dens = np.genfromtxt(data_path + data_name ,dtype='float64')

    dens_3d = dens.reshape([101,101,101])
    dens_3d = dens_3d[0:100 ,0:100, 0:100]


#     smooth_dens = smooting_3d(dens_3d,4)
#     smooth_dens = smooth_dens[2:100-2,2:100-2,2:100-2]

    if not os.path.isdir(dens_smooth_path_5 + str(data_name[:-4]) + '/'):
         os.makedirs(dens_smooth_path_5 + str(data_name[:-4]) + '/')
    os.chdir(dens_smooth_path_5 + str(data_name[:-4]) + '/')            

    for nn in range(2*box_size-4):
        plt.figure(figsize=[30,30])
        plt.imsave(str(nn) +'.png',np.log10(dens_3d[:,:,nn]),vmin=0,vmax=4,cmap='gray',dpi=300,format='png')
